In [2]:
from qiskit import QuantumCircuit
from qiskit.aqua.utils import get_entangler_map, validate_entangler_map

In [ ]:
class Count:
    def __init__(self, start_at=0):
        self._count = start_at
        
    @property
    def next(self):
        current = self._count
        self._count += 1
        return current

class RotationLayer:
    def __init__(self, count, gates):
        self._gates = gates
        self._count = count
        
    def construct_circuit(self, num_qubits):
        # if the input is a QuantumCircuit, just return that
        if isinstance(self._gates, QuantumCircuit):
            if self._gates.n_qubits != num_qubits:
                raise AquaError("Number of qubits not equals width of `gates`")
            return self._gates
        
        # if it's a string, set all gates to that string
        if isinstance(self._gates, str):
            self._gates = [self._gates] * num_qubits
            
        # if it's a list (of strings), set the gates to the strings
        if isinstance(self._gates, list):
            # make sure all elements are string
            if not all(isinstance(el, str) for el in self._gates):
                raise AquaError("All gate elements should be strings")
            
            # build circuit
            self._qc = QuantumCircuit(num_qubits)
            for qubit_id, gate in enumerate(self._gates):
                self.add_gate(gate, qubit_id)
        
            return self._qc
    
        # base case
        raise AquaError("Unsupported type of `gates`")
                
    def add_gate(self, gate, qubit):
        # gate without parameter
        if gate in ["x", "y", "z", "h"]:
            getattr(self._qc, gate)(qubit)
            
        elif gate in ["rx", "ry", "rz"]:
            param_number = self._count.next
            angle = Parameter(str(param_number))
            getattr(self._qc, gate)(angle, qubit)

        else:
            raise AquaError("Unsupported gate")
    
class EntanglementLayer:
    def __init__(self, count, gates="cz", entanglement="linear", entangler_map=None):
        self._gates = gates
        self._count = count
        if entangler_map is None:
            self._entangler_map = get_entangler_map(entanglement, num_qubits)
        else:
            self._entangler_map = validate_entangler_map(entangler_map, num_qubits)
        
    def construct_circuit(self, num_qubits):
        # if the input is a QuantumCircuit, just return that
        if isinstance(self._gates, QuantumCircuit):
            if self._gates.n_qubits != num_qubits:
                raise AquaError("Number of qubits not equals width of `gates`")
            return self._gates
        
        # if it's a string, set all gates to that string
        if isinstance(self._gates, str):
            self._gates = [self._gates] * len(self._entangler_map)
            
        # if it's a list (of strings), set the gates to the strings
        if isinstance(self._gates, list):
            # make sure all elements are string
            if not all(isinstance(el, str) for el in self._gates):
                raise AquaError("All gates should be strings")
            
            # build circuit
            self._qc = QuantumCircuit(num_qubits)
            for gate, (src, targ) in zip(self._gates, self._entangler_map):
                add_gate(gate, src, targ)
        
            return self._qc
        
        # base case
        raise AquaError("Unsupported type of `gates`")
        
    def add_gate(self, gate, src, targ):
        # gate without parameter
        if gate in ["cx", "cy", "cx", "ch"]:
            getattr(self._qc, gate)(src, targ)
            
        elif gate in ["crx", "cry", "crz"]:
            param_number = self._count.next
            angle = Parameter(str(param_number))
            getattr(self._qc, gate)(angle, src, targ)

        else:
            raise AquaError("Unsupported gate")


class VariationalForm:
    def __init__(self, 
                 num_qubits, 
                 rotation_gates,
                 entanglement_gates,
                 depth=3, 
                 entangler_map=None, 
                 entanglement='full', 
                 initial_state=None,
                 skip_unentangled_qubits=False,
                 skip_final_ry=False):
        """Constructs the parametrised Variational Form. 
        """
        
        # store parameters
        self._num_qubits = num_qubits
        self._depth = depth
        self._initial_state = initial_state
        self._skip_unentangled_qubits = skip_unentangled_qubits
        
        # get the rotation and entanglement layer of correct width
        # note, that to get the order of the parameters right, we have to call
        # `construct_circuit` in the correct order!
        parameter_count = Count()
        rotation_layer = RotationLayer(count, rotation_gates)
        entanglement_layer = EntanglementLayer(count, entanglement_gates, entanglement, entangler_map)
        
        # build parametrised circuit
        self._qc = QuantumCircuit(num_qubits)
        for i in range(depth):
            self._qc.append(rotation_layer.construct_circuit(num_qubits))
            self._qc.append(entanglement_layer.construct_circuit(num_qubits))
            
        if not skip_final_ry:
            self._qc.append(rotation_layer.construct_circuit(num_qubits))
        
        # store number of parameters
        self._num_parameters = len(self._qc.parameters)
        
            
    def construct_circuit(self, parameter_values):
        # check the number of provided parameters
        if len(parameter_values) != self._num_parameters:
            raise AquaError("Number of provided parameters doens't fit the number of circuit parameters.")
            
        # bind parameters            
        parameter_dict = {self._qc.parameters[i] : parameter_values[i]}
        bound_qc = self._qc.bind_parameters(parameter_dict)
        
        # return circuit with parameters
        return bound_qc
            
        

        
        

        